In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import ast

In [2]:
df = pd.read_csv('propertyGuru.csv',index_col=0)
df = df.dropna().reset_index(drop=True)

In [3]:
# # Filter Price:
def parsePrice(df):
    pattern = r"\d.*\d"
    for x in range(len(df)):
        try:
            price = re.search(pattern,df.at[x,'Price']).group().replace(',',"")
            df.at[x,'Price'] = int(price)
        except:
            print("Error")
    return df

In [4]:
# Filter MRT name from 
def parseMRT(df):
    df['DistanceFromMRT'] = None
    df['NearestMRT'] = None
    pattern = r"from .*"
    for x in range(len(df)):
        if df.at[x,'MRT'] == None:
            continue
        distance = re.search(r"\(.*\)",df.at[x,'MRT']).group().replace("(","").replace(")","").split(" ")
        try:
            if distance[1] == 'm':
                df.at[x,'DistanceFromMRT'] = int(distance[0])
            else:
                df.at[x,'DistanceFromMRT'] = int(float(distance[0])*1000)
        except:
            pass
        mrt = re.search(pattern,df.at[x,'MRT']).group().replace("from","").replace("MRT","").strip()
        df.at[x,'NearestMRT'] = mrt
    return df

In [5]:
df = parseMRT(df)
df = parsePrice(df)

Error
Error
Error


In [6]:
# Checkpoint 1
df.to_csv('test.csv')

# CheckPoint 1

In [7]:
df = pd.read_csv('test.csv',index_col=0)

df = df.drop_duplicates(subset=['ListID'])
df = df.reset_index(drop=True)

In [8]:
# Parsing Amenities:
def splitAmenities(dataFrame):
    dataFrame['NumberOfBeds'] = None
    dataFrame['NumberOfBaths'] = None
    dataFrame['Furnished'] = None
    dataFrame['Size'] = None
    bedPattern = r"\b\d+ bed\b|\bRoom\b"
    bathPattern = r"\b\d+ bath\b"
    sizePattern =  r"\d*,?\d+ sqft"
    furnishedPattern = r"\b\w+ Furnished\b"
    for x in range(len(dataFrame)):
        amenities = dataFrame.at[x,'Amenities']
        beds = re.search(bedPattern,amenities)
        if beds != None:
            dataFrame.at[x,'NumberOfBeds'] = beds.group()
            
        baths= re.search(bathPattern,amenities)
        if baths != None:
            dataFrame.at[x,'NumberOfBaths'] = baths.group()
        
        size = re.search(sizePattern,amenities)
        if size != None:
            dataFrame.at[x,'Size'] = size.group().replace('sqft','').rstrip()

        furnished = re.search(furnishedPattern,amenities)
        if furnished != None:
            dataFrame.at[x,'Furnished'] = furnished.group()
    return dataFrame

In [9]:
# Parsing Property Type:
def splitPropertyType(dataFrame):
    dataFrame['PropertyType'] = None
    
    bedPattern = r"\b\d+ bed\b|\bRoom\b"
    bathPattern = r"\b\d+ bath\b"
    sizePattern =  r"\d*,?\d+ sqft"
    furnishedPattern = r"\b\w+ Furnished\b"
    for x in range(len(dataFrame)):
        data = dataFrame.at[x,'Other Data']
        beds = re.search(bedPattern,data)
        if beds != None:
            dataFrame.at[x,'NumberOfBeds'] = beds.group()
            
        baths= re.search(bathPattern,data)
        if baths != None:
            dataFrame.at[x,'NumberOfBaths'] = baths.group()
        
        size = re.search(sizePattern,data)
        if size != None:
            dataFrame.at[x,'Size'] = size.group().replace('sqft','').rstrip()

        furnished = re.search(furnishedPattern,data)
        if furnished != None:
            dataFrame.at[x,'Furnished'] = furnished.group()
    return dataFrame

In [10]:
def splitPropertyType(dataFrame):
    dataFrame['PropertyType'] = None
    dataFrame['Floor Level'] = None
    dataFrame['PSF'] = None
    dataFrame['Tenure'] = None
    for x in range(len(df)):
        data = ast.literal_eval(dataFrame.at[x,'Other Data'])
        if "Property Type" in data:
            dataFrame.at[x,'PropertyType'] = data['Property Type'].replace("For Rent","").rstrip()
        
        if "Floor Level" in data:
            dataFrame.at[x,'Floor Level'] = data['Floor Level']

        if "PSF" in data:
            dataFrame.at[x,'PSF'] = data['PSF'].replace("psf","").replace("S$","").rstrip()

        if "Furnishing" in data and dataFrame.at[x,'Furnished'] == None:
            dataFrame.at[x,'Furnished'] = data['Furnishing']
    
        if "Tenure" in data:
            dataFrame.at[x,'Tenure'] = data['Tenure']
    return dataFrame


In [11]:
df = splitAmenities(df)
df = splitPropertyType(df)

In [12]:
df = df.dropna()
df = df.reset_index(drop=True)
df.to_csv("propertyGuru_filtered.csv")

# Checkpoint 2

In [13]:
df = pd.read_csv("propertyGuru_filtered.csv",index_col = 0)

In [15]:
# Taking out 'bed' and 'bath', etc
def filterNumbers(df):
    for x in range(len(df)):
        if df.at[x,'NumberOfBeds'] != 'Room' and df.at[x,'NumberOfBeds'] != None:
            df.at[x,'NumberOfBeds'] = int(df.at[x,'NumberOfBeds'].replace('bed',"").strip())
        if df.at[x,'NumberOfBaths'] != None:
            df.at[x,'NumberOfBaths'] = int(df.at[x,'NumberOfBaths'].replace('bath',"").strip())
        if df.at[x,'Size'] != None:
            df.at[x,'Size'] = int(df.at[x,'Size'].replace(',',"").strip())
    return df

In [16]:
df = filterNumbers(df)

In [17]:
df.to_csv('propertyGuru_completed.csv')

# Checkpoint 3

In [18]:
df

,ListID,Title,Price,MRT,Address,Amenities,Other Data,DistanceFromMRT,NearestMRT,NumberOfBeds,NumberOfBaths,Furnished,Size,PropertyType,Floor Level,PSF,Tenure
0,25202712.0,Meyer Mansion,8000,8 mins (630 m) from TE24 Katong Park MRT,79 Meyer Road East Coast / Marine Parade (D15),"['3 bed', '2 bath', '1,109 sqft', 'Partially F...","{'Property Type': 'Condominium For Rent', 'Flo...",630.0,TE24 Katong Park,3,2,Partially Furnished,1109,Condominium,High Floor,7.21,Freehold
1,25322701.0,749 Jurong West Street 73,4950,1 min (30 m) from JW1 Gek Poh MRT,749 Jurong West Street 73 Boon Lay / Jurong / ...,"['4 bed', '2 bath', '1,313 sqft', 'Fully Furni...","{'Property Type': '5I HDB For Rent', 'Floor Si...",30.0,JW1 Gek Poh,4,2,Fully Furnished,1313,5I HDB,Middle Floor,3.77,99-year Leasehold
2,25226930.0,112 Bishan Street 12,4050,9 mins (730 m) from NS17/CC15 Bishan MRT,112 Bishan Street 12 Ang Mo Kio / Bishan / Tho...,"['3 bed', '2 bath', '1,130 sqft', 'Fully Furni...","{'Property Type': '4A HDB For Rent', 'Floor Si...",730.0,NS17/CC15 Bishan,3,2,Fully Furnished,1130,4A HDB,High Floor,3.58,99-year Leasehold
3,19207068.0,Aspen Heights,15000,14 mins (1.2 km) from DT20 Fort Canning MRT,261 River Valley Road Orchard / River Valley (...,"['5 bed', '4 bath', '2,700 sqft', 'Partially F...","{'Property Type': 'Condominium For Rent', 'Flo...",1200.0,DT20 Fort Canning,5,4,Partially Furnished,2700,Condominium,Penthouse,5.56,999-year Leasehold
4,25101247.0,Reflections at Keppel Bay,2500,14 mins (1.2 km) from CC28 Telok Blangah MRT,1 Keppel Bay View Harbourfront / Telok Blangah...,"['Room', '1 bath', '170 sqft', 'Partially Furn...","{'Property Type': 'Condominium For Rent', 'Flo...",1200.0,CC28 Telok Blangah,Room,1,Partially Furnished,170,Condominium,High Floor,14.71,99-year Leasehold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16844,24623132.0,Hillcrest Arcadia,6900,24 mins (2 km) from DT8 Tan Kah Kee MRT,255 Arcadia Road Newton / Novena (D11),"['3 bed', '3 bath', '1,798 sqft', 'Partially F...","{'Property Type': 'Condominium For Rent', 'Flo...",2000.0,DT8 Tan Kah Kee,3,3,Partially Furnished,1798,Condominium,High Floor,3.84,99-year Leasehold
16845,25305073.0,Double Bay Residences,2500,7 mins (560 m) from EW3 Simei MRT,17 Simei Street 4 Pasir Ris / Tampines (D18),"['1 bed', '1 bath', '350 sqft', 'Fully Furnish...","{'Property Type': 'Condominium For Rent', 'Flo...",560.0,EW3 Simei,1,1,Fully Furnished,350,Condominium,Middle Floor,7.14,99-year Leasehold
16846,25316743.0,The Armadale,4400,8 mins (690 m) from NS20 Novena MRT,60 Gilstead Road Newton / Novena (D11),"['2 bed', '1 bath', '1,184 sqft', 'Partially F...","{'Property Type': 'Condominium For Rent', 'Flo...",690.0,NS20 Novena,2,1,Partially Furnished,1184,Condominium,Penthouse,3.72,Freehold
16847,18423872.0,Emery Point,5300,12 mins (1 km) from CC8 Dakota MRT,18 Ipoh Lane East Coast / Marine Parade (D15),"['3 bed', '3 bath', '1,302 sqft', 'Fully Furni...","{'Property Type': 'Condominium For Rent', 'Flo...",1000.0,CC8 Dakota,3,3,Fully Furnished,1302,Condominium,High Floor,4.07,Freehold
